<a href="https://colab.research.google.com/github/aliprg0/Google_Colab/blob/main/CH_G12T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

In [4]:
clmns = [
       "Open 1","Close 1","Open 2","Close 2","Open 3","Close 3","Open 4","Close 4","Open 5","Close 5","Open 6","Close 6","Open 7","Close 7","Open 8","Close 8","Open 9","Close 9","Open 10","Close 10","Open 11","Close 11","Open 12","Close 12","Open 13","Close 13","Open 14","Close 14","Open 15","Close 15",

    "suggestion"]
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["buy","sell"], axis=1)
  y = data[["buy","sell"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-10,10))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(30, data.shape[0]-1):
        grow = []
        ggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][3]])
        sugg = "sell"
        if data[i][3] > data[i-1][3]:
            sugg = "buy"
        arr = np.array(grow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        arr = np.append(arr, sugg)
        row.append(arr)
    grow = []
    ggrow = []
    gggrow = []
    arr = []
    return np.array(row)

In [2]:
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
           if data.shape[0] > 30:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 40 == 0:
          print(" ")
        address = f"/content/data/{file}"
        try:
           unattached_dfs.append(process(pd.read_csv(address)))
        except:
          print(" EP ")
     if np.array(unattached_dfs[0]).shape[0] == 0:
            print(" Null ")
            data = np.array(unattached_dfs[1])
            for z in unattached_dfs[2:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     else:
            data = np.array(unattached_dfs[0])
            for z in unattached_dfs[1:]:
               try: 
                  data = np.append(data, np.array(z), axis=0)
               except:
                  pass
     unattached_dfs = []
     data = pd.DataFrame(data, columns=clmns)
     sugg = data["suggestion"]
     data.drop("suggestion",axis=1,inplace=True)
     sugg = pd.get_dummies(sugg)
     data = pd.concat([data,sugg],axis=1)
     data = data.astype(float)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces):
  pd.options.mode.chained_assignment = None
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  
  index = 0 
  for files in new_files:
     
     each_file_proc(files,now,index)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")      

In [8]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"730d","1h")

Symbols : 1500
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [ ]:
folder_name = extract_data(500)
to_par(folder_name,10)
xTrain,xTest,yTrain,yTest = make_df(folder_name)

Files In Data : 1499
Processing File:
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40  
41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80  
81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116  EP 
117 118 119 120  
121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152  EP 
153 154  EP 
155 

In [7]:
xTrain,xTest,yTrain,yTest = spliting(pd.read_parquet("/content/drive/MyDrive/CD.parquet"))

(1607157, 30) (1607157, 2)
(178573, 30) (178573, 2)


In [12]:
model = Sequential()

model.add(Dense(750, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))
model.add(Dense(750, activation='relu'))


opt = tf.keras.optimizers.Adamax()
model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 750)               23250     
                                                                 
 dense_7 (Dense)             (None, 750)               563250    
                                                                 
 dense_8 (Dense)             (None, 750)               563250    
                                                                 
 dense_9 (Dense)             (None, 750)               563250    
                                                                 
 dense_10 (Dense)            (None, 750)               563250    
                                                                 
 dense_11 (Dense)            (None, 750)               563250    
                                                                 
 dense_12 (Dense)            (None, 2)                

In [14]:
model.fit(xTrain,yTrain,epochs=10,batch_size=10000,validation_data=(xTest,yTest))

Epoch 1/10
161/161 [==============================] - 19s 114ms/step - loss: 0.0503 - accuracy: 0.9824 - val_loss: 0.1910 - val_accuracy: 0.9538
Epoch 2/10
161/161 [==============================] - 18s 114ms/step - loss: 0.0369 - accuracy: 0.9862 - val_loss: 0.1968 - val_accuracy: 0.9539
Epoch 3/10
161/161 [==============================] - 18s 114ms/step - loss: 0.0350 - accuracy: 0.9866 - val_loss: 0.2034 - val_accuracy: 0.9533
Epoch 4/10
161/161 [==============================] - 18s 114ms/step - loss: 0.0354 - accuracy: 0.9863 - val_loss: 0.2056 - val_accuracy: 0.9533
Epoch 5/10
161/161 [==============================] - 18s 114ms/step - loss: 0.0362 - accuracy: 0.9861 - val_loss: 0.2119 - val_accuracy: 0.9524
Epoch 6/10
161/161 [==============================] - 19s 118ms/step - loss: 0.0382 - accuracy: 0.9855 - val_loss: 0.2176 - val_accuracy: 0.9520
Epoch 7/10
161/161 [==============================] - 19s 117ms/step - loss: 0.0405 - accuracy: 0.9848 - val_loss: 0.2165 - val_ac

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model.save("CMC_G115_9538.h5")

In [17]:
yf.download("btc-usd",period="5d",interval="1h")

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2022-05-03 00:00:00+00:00,38528.109375,38554.058594,38429.773438,38432.628906,38432.628906,0
2022-05-03 01:00:00+00:00,38430.406250,38504.238281,38381.968750,38493.644531,38493.644531,0
2022-05-03 02:00:00+00:00,38496.835938,38570.828125,38371.152344,38423.109375,38423.109375,0
2022-05-03 03:00:00+00:00,38418.949219,38574.425781,38411.089844,38503.128906,38503.128906,160819200
2022-05-03 04:00:00+00:00,38505.218750,38539.828125,38476.527344,38536.054688,38536.054688,0
...,...,...,...,...,...,...
2022-05-07 10:00:00+00:00,35999.835938,36096.949219,35978.820312,36058.574219,36058.574219,0
2022-05-07 11:00:00+00:00,36067.761719,36129.925781,36045.937500,36049.042969,36049.042969,0
2022-05-07 12:00:00+00:00,36049.160156,36057.148438,35989.609375,36017.605469,36017.605469,0
2022-05-07 13:00:00+00:00,36021.976562,36040.406250,35913.957031,35913.957031,35913.957031,0


In [40]:
def process_for_prediction(data):
        i = -14
        if len(pd.DataFrame(data).columns) == 7:
          data = data.iloc[: , 1:]        
        data = np.array(data)
        grow = []
        ggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][3]])
        arr = np.array(grow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        return arr
def make_prediction(symbol,period,timeframe):
    raw_data = process_for_prediction(yf.download(symbol,period=period,interval=timeframe))
    return  model.predict(np.array(raw_data).reshape(1,-1))

In [41]:
make_prediction("btc-usd","5d","1h")

[*********************100%***********************]  1 of 1 completed


array([[0.97238106, 0.02279101]], dtype=float32)

In [ ]:
def process_for_prediction(data):
        i = -1
        row = []
        data.drop("symbol",axis=1,inplace=True)   
        data = np.array(data)
        grow = []
        ggrow = []
        for x in range(1, 31):
            grow.append([data[i-x][3]])
        arr = np.array(grow).flatten()
        arr = scaler(arr.reshape(-1, 1))
        return arr
def make_prediction(symbol,timeframe):
   tv = TvDatafeed()
   data = tv.get_hist(symbol='btcUSDt',exchange='binance',interval=Interval.in_monthly,n_bars=100)
   return model.predict(np.array(process_for_prediction(data)).reshape(1,-1))

In [ ]:
make_prediction("s","x")

array([[0.00273842, 0.9986447 ]], dtype=float32)

In [ ]:
tv = TvDatafeed()
tv.get_hist(symbol='btcUSD',exchange='bitstamp',interval=Interval.in_monthly,n_bars=100)



do you want to install chromedriver automatically?? y/n	n


you are using nologin method, data you access may be limited


,symbol,open,high,low,close,volume
datetime,,,,,,
2014-02-01,bitstamp:btcUSD,803.00,827.38,400.00,550.10,824911.000000
2014-03-01,bitstamp:btcUSD,551.80,710.00,436.00,454.83,491824.650960
2014-04-01,bitstamp:btcUSD,454.81,548.00,339.79,448.85,515027.729378
2014-05-01,bitstamp:btcUSD,449.00,629.40,420.27,627.80,300028.000000
2014-06-01,bitstamp:btcUSD,627.80,683.26,538.38,641.11,288061.000000
...,...,...,...,...,...,...
2022-01-01,bitstamp:btcUSD,46230.00,47989.00,32950.72,38491.92,71879.817236
2022-02-01,bitstamp:btcUSD,38483.56,45850.00,34324.05,43178.98,57987.188733
2022-03-01,bitstamp:btcUSD,43221.71,48234.00,37169.52,45517.27,57990.054358
